In [2]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [4]:

from notebook_utils import NotebookUtilities
import os.path as osp

nu = NotebookUtilities(data_folder_path=osp.abspath('../data'), saves_folder_path=osp.abspath('../saves'))

In [ ]:

table_dict = nu.get_page_tables(str(table_soup), verbose=False)[0].to_dict(orient='list')
if 'Names or topics' in table_dict.keys(): topics_list = table_dict['Names or topics']
elif 'Persons or topics' in table_dict.keys(): topics_list = table_dict['Persons or topics']
else: print(table_dict.keys()); raise
for date_str, topic_str in zip(table_dict['Date'], topics_list):
    cover_topic_str = re.sub(r', *Sr\.', '_ Sr.', cover_topic)
    cover_topic_str = re.sub(r', *Duke', '_ Duke', cover_topic_str)
    cover_topics_list = re.split(', *', cover_topic_str, 0)
    for cover_topic_str in cover_topics_list:
        cover_topic_str = cover_topic_str.split('[')[0].replace('_', ',')
        if (cover_topic_str == 'Queen Marie'): cover_topic_str = 'Marie of Romania'
        elif (cover_topic_str == 'Queen Marie'): cover_topic_str = 'Marie of Romania'

In [ ]:

import os

exe_list = ['wc', 'gvpr']
for sub_directory, directories_list, files_list in os.walk(r'D:\\'):
    for file_name in files_list:
        if any(map(lambda x: file_name == f'{x}.exe', exe_list)):
            file_path = os.path.join(sub_directory, file_name)
            print(f'{file_path}')

In [ ]:

%run ../load_magic/storage.py
%run ../load_magic/dataframes.py
%run ../load_magic/paths.py
%run ../load_magic/lists.py
%run ../load_magic/environment.py
%pprint
notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
[fn for fn in dir() if not fn.startswith('_')]

In [ ]:

                try:
                except Exception as e:
                    print(f'{e.__class__} exception at peakutils.indexes({y}, thres={thres}, min_dist={min_dist}): {str(e).strip()}')

In [17]:

rest_times_df = DataFrame([])
runners_dict = {
    'amanda': 'Amanda Nelson', 'bartosz': 'Bartosz Fudali', 'ihor': 'Ihor Verys', 'keith': 'Keith Russell', 'merijn': 'Merijn Geerts', 'phil': 'Phil Gore', 'sam': 'Sam Harvey',
    'terumichi': 'Terumichi Morishita'
}
for sub_directory, directories_list, files_list in os.walk(r'C:\Users\daveb\Downloads'):
    for file_name in files_list:
        if file_name.endswith('.html') and ('homepage' not in file_name):
            file_path = osp.join(sub_directory, file_name)
            rest_times_df = pd.concat(
                [rest_times_df, get_single_runner_rest_times_data_frame(table_path=file_path, runner_name=runners_dict[file_name.split('.')[0]])], axis='index'
            )
rest_times_df.sample(min(16, rest_times_df.shape[0])).dropna(axis='columns', how='all').T

,72,41,88,56,87,70,36,28,91,60,79,0,25,4,58,61
#,73,42,89,57,88,71,37,29,92,61,80,1,26,5,59,62
Measurement,79:47:58,48:43:22,95:44:59,63:55:28,94:48:10,77:47:34,43:40:44,35:50:34,98:49:57,67:46:47,86:52:13,7:53:36,32:53:33,11:37:07,65:56:52,68:39:58
Lap Split,47:58,43:22,44:59,55:28,48:10,47:34,40:44,50:34,49:57,46:47,52:13,53:36,53:33,37:06,56:52,39:58
Rest Time,721,997,900,271,709,745,1155,565,602,792,466,383,386,1373,187,1201
runner_name,Sam Harvey,Amanda Nelson,Ihor Verys,Amanda Nelson,Terumichi Morishita,Merijn Geerts,Keith Russell,Terumichi Morishita,Phil Gore,Bartosz Fudali,Bartosz Fudali,Ihor Verys,Ihor Verys,Bartosz Fudali,Merijn Geerts,Phil Gore


In [ ]:

print()
for i in range(0, 180, 20):
    print(f'''    <path
       class="tick-style major-tick"
       d="m {1.56 * i + 0.825315},161.4459 v 13.10819"
       id="path-millennium-tick{i:03}"
       style="stroke-opacity:1;stroke-width:0.79375;stroke:#160f20;"
    />''')
    # print(f'''    <path
    #    class="tick-style minor-tick"
    #    d="m {1.56 * i + 0.825315},163.31371 v 9.37257"
    #    id="path-quincentenary-tick{i:03}"
    #    style="stroke-opacity:1;stroke:#160f20;"
    # />''')
    # print(f'''    <path
    #    class="century-tick"
    #    d="m {1.56 * i + 0.825315},164.825 v 6.35"
    #    id="path-century-tick{i:03}" />''')


    <path
       class="tick-style major-tick"
       d="m 0.825315,161.4459 v 13.10819"
       id="path-millennium-tick000"
       style="stroke-opacity:1;stroke-width:0.79375;stroke:#160f20;"
    />
    <path
       class="tick-style major-tick"
       d="m 32.025315000000006,161.4459 v 13.10819"
       id="path-millennium-tick020"
       style="stroke-opacity:1;stroke-width:0.79375;stroke:#160f20;"
    />
    <path
       class="tick-style major-tick"
       d="m 63.22531500000001,161.4459 v 13.10819"
       id="path-millennium-tick040"
       style="stroke-opacity:1;stroke-width:0.79375;stroke:#160f20;"
    />
    <path
       class="tick-style major-tick"
       d="m 94.42531500000001,161.4459 v 13.10819"
       id="path-millennium-tick060"
       style="stroke-opacity:1;stroke-width:0.79375;stroke:#160f20;"
    />
    <path
       class="tick-style major-tick"
       d="m 125.62531500000001,161.4459 v 13.10819"
       id="path-millennium-tick080"
       style="stroke-opacity:1;s